In [13]:
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from auxiliary_functions import create_position_planes,encode_move
import tqdm
import chess
from tqdm import tqdm
import pandas as pd

Load the data:

In [ ]:
'''
So the general idea: label positions, based on the next best tactical move.
'''

In [66]:
pieces = {
    "k": 0,
    "q": 1,
    "r": 2,
    "b": 3,
    "n": 4,
    "p": 5
}

In [67]:
# This is for labels and data:
label_pieces = []
label_move_types = []
positions = []


file_path = r'C:\Users\csata\OneDrive\Desktop\FÖT\Chess_Complexity\data\lichess_db_puzzle.csv\lichess_db_puzzle.csv'

df = pd.read_csv(file_path)

it = 0

tqdm.pandas()
for index, row in tqdm(df.iterrows(), total=len(df)):
    if it >= 100000:
        break
    it += 1

    # This part is for creating the 14 planes which reresents the position:
    curr_position = row['FEN']
    position_planes = create_position_planes(curr_position)
    positions.append(position_planes)
    # In this part I save the labeles according to the type of movement(plane for direction and distance travelled):
    curr_move = row['Moves'].split(' ')[0]
    temp_move_plane = encode_move(curr_move)
    
    #temp_move_types = np.zeros((64,1))
    #temp_move_types[temp_move_plane] += 1
    label_move_types.append(temp_move_plane)
    
    # In this part I save the labels according to which piece was moved in the given position:
    curr_board = chess.Board(curr_position)
    curr_piece = curr_board.piece_at(chess.parse_square(curr_move[:2]))
    
    #temp_pieces = np.zeros((6,1))
    #temp_pieces[pieces[curr_piece.__str__().lower()]] += 1
    label_pieces.append(pieces[curr_piece.__str__().lower()])




  2%|▏         | 100000/4211138 [00:53<36:42, 1866.75it/s]


In [68]:

def Fisher_train(positions,labels):
    
    nsamples, nx, ny,nz = positions.shape
    X = positions.reshape((nsamples,nx*ny*nz))

    print(f'This is shape of X: {X.shape}')

    # Split into train/test sets
    X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

    # Standardize the data (important for LDA)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Apply LDA (Linear Discriminant Analysis) to reduce dimensions and create Fisherfaces-like features
    lda = LinearDiscriminantAnalysis(n_components=None)  # None will keep the maximum components
    X_train_lda = lda.fit_transform(X_train_scaled, y_train)
    X_test_lda = lda.transform(X_test_scaled)

    # Use a classifier on the LDA-transformed data
    # Let's use an SVM for classification (could use other classifiers like k-NN)
    clf = SVC(kernel='linear')
    clf.fit(X_train_lda, y_train)

    # Evaluate the model
    accuracy = clf.score(X_test_lda, y_test)
    print(f"Test accuracy: {accuracy:.4f}")

print('This is result for label_pieces:')
Fisher_train(np.asarray(positions),np.asarray(label_pieces))

print('This is result for label_move_types:')
Fisher_train(np.asarray(positions),np.asarray(label_move_types))



This is result for label_pieces:
This is shape of X: (100000, 896)
Test accuracy: 0.3585
This is result for label_move_types:
This is shape of X: (100000, 896)
Test accuracy: 0.1638


In [78]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Define a simple neural network (MLP)
class SimpleNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)  
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(512, 256)          
        self.fc3 = nn.Linear(256, 128)  
        self.fc4 = nn.Linear(128, num_classes)  # Output layer

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        
        x = self.fc4(x)  # No activation on final layer (logits)
        return x

def Fisher_train(positions, labels,y_type):
    train_losses = []
    accuracies = []

    # Check if GPU is available and use it
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'Using device: {device}')

    # Reshape data
    nsamples, nx, ny, nz = positions.shape
    X = positions.reshape((nsamples, nx * ny * nz))

    print(f'This is shape of X: {X.shape}')

    # Split into train/test sets
    X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

    # Standardize the data (important for neural networks)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Convert data to PyTorch tensors
    X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)
    X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)

    # Create datasets and dataloaders
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # Define model, loss function, and optimizer
    input_size = X_train_scaled.shape[1]  # Number of input features
    if y_type:
        num_classes = 6  # Number of classes (labels)
    else:
        num_classes = 64  # Number of classes (labels)

    model = SimpleNN(input_size, num_classes).to(device)  # Send model to GPU

    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total number of trainable parameters: {total_params}")

    criterion = nn.CrossEntropyLoss()  # For multi-class classification
    optimizer = optim.SGD(model.parameters(), lr=0.01)

    # Training loop
    num_epochs = 20
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(train_loader):.4f}")
        train_losses.append(running_loss / len(train_loader))

        # Evaluation on test set
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                outputs = model(X_batch)
                _, predicted = torch.max(outputs, 1)
                total += y_batch.size(0)
                correct += (predicted == y_batch).sum().item()

        accuracy = correct / total
        accuracy = accuracy * 100
        print(f"Test accuracy: {accuracy:.4f}%")
        accuracies.append(accuracy)


# Example usage:
print('This is result for label_pieces:')
label_losses, label_accruacies = Fisher_train(np.asarray(positions), np.asarray(label_pieces),True)

print('This is result for label_move_types:')
move_type_losses, move_type_accuracies = Fisher_train(np.asarray(positions), np.asarray(label_move_types),False)

import matplotlib.pyplot as plt
def plot_loss_and_accuracy(losses, accuracies,name="training_loss_and_accuracy.png"):
    epochs = range(1, len(losses) + 1)

    # Create a figure with two subplots (1 row, 2 columns)
    fig, ax1 = plt.subplots(figsize=(10,5))

    # Plot Loss
    ax1.plot(epochs, losses, 'b-', label='Training Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss', color='b')
    ax1.tick_params('y', colors='b')

    # Create a twin axis for accuracy
    ax2 = ax1.twinx()
    ax2.plot(epochs, accuracies, 'r-', label='Test Accuracy')
    ax2.set_ylabel('Accuracy', color='r')
    ax2.tick_params('y', colors='r')

    # Add a title
    plt.title('Training Loss and Test Accuracy over Epochs')

    # Save the figure as a PNG file
    plt.savefig(name)

    # Show the plot
    plt.show()


plot_loss_and_accuracy(label_losses, label_accruacies,name="move_types.png")
plot_loss_and_accuracy(move_type_losses, move_type_accuracies,name="move_types.png")


This is result for label_pieces:
Using device: cpu
This is shape of X: (100000, 896)
Total number of trainable parameters: 624262
Epoch 1/20, Loss: 1.7248
Test accuracy: 29.8350%
Epoch 2/20, Loss: 1.6102
Test accuracy: 32.0400%
Epoch 3/20, Loss: 1.5273
Test accuracy: 34.4900%
Epoch 4/20, Loss: 1.4394
Test accuracy: 35.6800%
Epoch 5/20, Loss: 1.3579
Test accuracy: 36.2800%
Epoch 6/20, Loss: 1.2658
Test accuracy: 36.2650%
Epoch 7/20, Loss: 1.1580
Test accuracy: 35.5500%
Epoch 8/20, Loss: 1.0371
Test accuracy: 34.4350%
Epoch 9/20, Loss: 0.9085
Test accuracy: 34.0150%
Epoch 10/20, Loss: 0.7796
Test accuracy: 34.9600%
Epoch 11/20, Loss: 0.6669
Test accuracy: 34.7400%
Epoch 12/20, Loss: 0.5635
Test accuracy: 33.5550%
Epoch 13/20, Loss: 0.4813
Test accuracy: 32.9100%
Epoch 14/20, Loss: 0.4023
Test accuracy: 33.3950%
Epoch 15/20, Loss: 0.3490
Test accuracy: 33.5350%
Epoch 16/20, Loss: 0.3025
Test accuracy: 33.3000%
Epoch 17/20, Loss: 0.2522
Test accuracy: 33.1300%
Epoch 18/20, Loss: 0.2192
Tes

TypeError: cannot unpack non-iterable NoneType object

In [74]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

model = SimpleNN()
total_params = count_parameters(model)
print(f"Total number of trainable parameters: {total_params}")


TypeError: SimpleNN.__init__() missing 2 required positional arguments: 'input_size' and 'num_classes'